In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder
import pickle

In [2]:
df = pd.read_csv("Data/autos.csv",header=0,sep=',',encoding='Latin1')

In [3]:
#print al the different sellers
print(df.seller.value_counts()) 
#remove the seller type having only 3 cars 
df[df.seller != 'gewerblich']
 #now all the sellers are same so we can get rid of this column 
df=df.drop('seller',1)
#print al the different sellers 
print(df.offerType.value_counts())
#remove the Offer Type havng only 12 listings 
df[df.offerType != 'Gesuch']
 #now all the offers are same so we can get rid of this column 
df=df.drop('offerType',1)

privat        371525
gewerblich         3
Name: seller, dtype: int64
Angebot    371516
Gesuch         12
Name: offerType, dtype: int64


C:\Users\Abijeeth\AppData\Local\Temp\ipykernel_1828\976391629.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df=df.drop('seller',1)
C:\Users\Abijeeth\AppData\Local\Temp\ipykernel_1828\976391629.py:12: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df=df.drop('offerType',1)


In [4]:
print(df.shape) 
df = df[(df.powerPS > 50) & (df.powerPS <900)]
print(df.shape) 
df = df[(df.yearOfRegistration >= 1950) & (df.yearOfRegistration < 2017)]
print(df.shape)

(371528, 18)
(319709, 18)
(309171, 18)


In [5]:
df.head()

,dateCrawled,name,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,BMW_316i___e36_Limousine___Bastlerfahrzeug__Ex...,650,test,limousine,1995,manuell,102,3er,150000,10,benzin,bmw,ja,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07


In [6]:
df.drop(['name', 'abtest', 'dateCrawled','nrOfPictures', 'lastSeen', 'postalCode', 'dateCreated'], axis='columns', inplace=True)

In [7]:
new_df = df.copy()
new_df = new_df.drop_duplicates(['price', 'vehicleType', 'yearOfRegistration','gearbox', 'powerPS', 'model', 'kilometer', 'monthOfRegistration', 'fuelType' , 'notRepairedDamage'])

In [8]:
new_df.gearbox.replace(('manuell', 'automatik'), ('manual','automatic'),inplace=True) 
new_df.fuelType.replace(('benzin', 'andere', 'elektro'), ('petrol', 'others', 'electric'),inplace=True)
new_df.vehicleType.replace(('kleinwagen', 'cabrio', 'kombi', 'andere'),('small car','convertible', 'combination', 'others'), inplace=True) 
new_df.notRepairedDamage.replace(('ja', 'nein'), ('Yes', 'No'), inplace =True)

In [9]:
new_df = new_df[(new_df.price >= 100) & (new_df.price <= 150000)]
new_df['notRepairedDamage'].fillna(value='not-declared', inplace=True)
new_df['fuelType'].fillna(value='not-declared', inplace=True)
new_df['gearbox'].fillna(value='not-declared', inplace=True) 
new_df['vehicleType'].fillna(value='not-declared', inplace=True)
new_df['model'].fillna(value='not-declared', inplace=True)

In [10]:
new_df.to_csv("autos_preprocessed.csv")

In [11]:
labels = ['gearbox', 'notRepairedDamage', 'model', 'brand', 'fuelType', 'vehicleType']
mapper = {}
for i in labels:
  mapper[i] = LabelEncoder()
  mapper[i].fit(new_df[i]) 
  tr = mapper[i].transform(new_df[i])
  np.save(str('classes'+i+'.npy'), mapper[i].classes_)
  print(i,":",mapper[i]) 
  new_df.loc[:, i + '_labels'] = pd. Series(tr, index=new_df.index)
#Final data to be put in a new dataframe called "LABELED", 
labeled = new_df[ ['price', 'yearOfRegistration' , 'powerPS' ,'kilometer' , 'monthOfRegistration']+ [x+"_labels" for x in labels]]
print(labeled.columns)

gearbox : LabelEncoder()
notRepairedDamage : LabelEncoder()
model : LabelEncoder()
brand : LabelEncoder()
fuelType : LabelEncoder()
vehicleType : LabelEncoder()
Index(['price', 'yearOfRegistration', 'powerPS', 'kilometer',
       'monthOfRegistration', 'gearbox_labels', 'notRepairedDamage_labels',
       'model_labels', 'brand_labels', 'fuelType_labels',
       'vehicleType_labels'],
      dtype='object')


In [12]:
Y = labeled.iloc[:,0].values
X = labeled.iloc[:,1:].values
Y = Y.reshape(-1,1)

In [13]:
labeled.head()

,price,yearOfRegistration,powerPS,kilometer,monthOfRegistration,gearbox_labels,notRepairedDamage_labels,model_labels,brand_labels,fuelType_labels,vehicleType_labels
1,18300,2011,190,125000,5,1,1,162,1,1,3
2,9800,2004,163,125000,8,0,2,118,14,1,8
3,1500,2001,75,150000,6,1,0,117,38,7,7
4,3600,2008,69,90000,7,1,0,102,31,1,7
5,650,1995,102,150000,10,1,1,11,2,7,4


In [14]:
from sklearn.model_selection import cross_val_score, train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X ,Y ,test_size = 0.3,random_state = 3)

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
regressor = RandomForestRegressor(n_estimators = 1000,max_depth=10,random_state=34)

In [16]:
regressor.fit(X_train,np.ravel(Y_train,order='C'))

RandomForestRegressor(max_depth=10, n_estimators=1000, random_state=34)

In [17]:
y_pred = regressor.predict(X_test)
print(r2_score(Y_test,y_pred))

0.834527626497731


In [19]:
filename = 'resale_model.sav'
pickle.dump(regressor,open(filename,'wb'))